In [143]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  # for plotting facilities

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# Any results you write to the current directory are saved as output.

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [144]:
test_data.head()
# train_data.isnull().sum()

,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy
0,524164,0.410013,-16.794967,0.023500,0.232910,0.012689,0.271585,0.664321,302901.5498,0.424867
1,524165,0.463071,-1.357000,0.141818,0.057725,0.257942,0.097624,0.829552,221995.6643,0.846000
2,524166,0.686569,-3.368928,0.167851,0.287823,0.210915,0.325909,0.304978,357724.0127,0.134067
3,524167,0.885793,-5.598049,0.118488,0.000005,0.376906,0.134435,0.487740,271790.3989,0.316467
4,524168,0.637391,-7.068160,0.126099,0.539073,0.068950,0.024300,0.591248,277728.5383,0.481067


In [145]:
X = train_data.drop(['BeatsPerMinute','id'], axis=1)
y = train_data['BeatsPerMinute']


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

In [146]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

data_dmatrix = xgb.DMatrix(data=X, label=y)

In [147]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import numpy as np

# (Optional) make a proper validation set for early stopping
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.3, random_state=13)

model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.1,
    early_stopping_rounds=50,
    eval_metric="rmse",
    max_depth=10,
    subsample=0.5,
    colsample_bytree=0.8,
    random_state=12
)

model.fit(
    X_tr, y_tr,
    eval_set=[(X_val, y_val)],
    verbose=False
)

pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, pred))
mae  = mean_absolute_error(y_test, pred)

print(f"Best iteration: {getattr(model, 'best_iteration', None)}")
print(f"RMSE: {rmse:.10f}")
print(f"MAE : {mae:.10f}")


Best iteration: 0
RMSE: 26.4965947946
MAE : 21.2252329046


In [148]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor

from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.size'] = 18
%matplotlib inline

N_FOLDS = 5
# MAX_EVALS = 5



In [149]:
features = pd.read_csv('train.csv')

# Sample 16000 rows (10000 for training, 6000 for testing)
features = features.sample(n = 16000, random_state = 42)

# Only numeric features
features = features.select_dtypes('number')

# Extract the labels
labels = np.array(features['BeatsPerMinute'].astype(np.int32)).reshape((-1, ))
features = features.drop(columns = ['BeatsPerMinute', 'id'])

# Split into training and testing data
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 6000, random_state = 42)

print('Train shape: ', train_features.shape)
print('Test shape: ', test_features.shape)

train_features.head()

Train shape:  (10000, 9)
Test shape:  (6000, 9)


,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy
420513,0.796931,-4.192561,0.055570,0.569120,0.000001,0.221924,0.390025,284478.7241,0.077333
429006,0.858477,-7.555815,0.034582,0.477072,0.184986,0.288436,0.596729,297568.3043,0.296067
466638,0.434569,-10.715466,0.066433,0.000005,0.000001,0.235537,0.908803,270687.6234,0.827267
352104,0.622971,-11.415735,0.023500,0.242030,0.162070,0.170701,0.481447,357866.4033,0.414867
33789,0.842075,-13.709634,0.052893,0.000005,0.455901,0.024300,0.681867,251568.2692,0.511867


In [ ]:
# baseline model

model = XGBRegressor()
model.fit(train_features, train_labels)

hyperparameters = model.get_xgb_params()

cross_val_scores = cross_val_score(model, train_features, train_labels, cv=N_FOLDS, scoring='neg_mean_squared_error')
print('The best RMSE with the default settings is {:.5f}'.format(-cross_val_scores.max()))


some_params = {'gamma': np.float64(6.071378912516411),
 'learning_rate': np.float64(0.012223406544371523),
 'max_depth': int(np.float64(11.0)),
 'n_estimators': int(np.float64(32.0)),
 'num_leaves': int(np.float64(140.0))}
optimized_model = XGBRegressor(**some_params)
optimized_model.fit(train_features, train_labels)
cross_val_scores = cross_val_score(optimized_model, train_features, train_labels, cv=N_FOLDS, scoring='neg_mean_squared_error')
print('The best RMSE after doing some bayesian optimization is {:.5f}'.format(-cross_val_scores.max()))





The best RMSE with the default settings is 795.31305


/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:15:42] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "num_leaves" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:15:42] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "num_leaves" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:15:43] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "num_leaves" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [20:15:44] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "num_leaves" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/steven/.pyenv/versions/.env/li

The best RMSE after doing some bayesian optimization is 672.27649


In [161]:
X_test = test_data.drop(['id'], axis=1)

X_test.head()

,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy
0,0.410013,-16.794967,0.023500,0.232910,0.012689,0.271585,0.664321,302901.5498,0.424867
1,0.463071,-1.357000,0.141818,0.057725,0.257942,0.097624,0.829552,221995.6643,0.846000
2,0.686569,-3.368928,0.167851,0.287823,0.210915,0.325909,0.304978,357724.0127,0.134067
3,0.885793,-5.598049,0.118488,0.000005,0.376906,0.134435,0.487740,271790.3989,0.316467
4,0.637391,-7.068160,0.126099,0.539073,0.068950,0.024300,0.591248,277728.5383,0.481067


In [162]:
# generate a submission


preds = optimized_model.predict(X_test)

In [176]:
# X_test['id'] = test_data['id']
submission = pd.DataFrame({'ID': test_data['id'], 'BeatsPerMinute': preds}, index=None)

submission.head()

submission.to_csv('submission.csv', index=False)

In [179]:
# The first step is to write an objective function that takes
# in a set of hyperparameters and returns the cross-validation
# score on the training data. We're using Hyperopt, which means
# an objective function must return either a single real value
# or a dictionary with a 'loss' key with the score to minimize,
# and a 'status' key indicating whether the run was successful.

ITERATION = 0
OUT_FILE = 'bayes_test.csv'


import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer

def objective(hyperparameters):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization.
    Writes a new line to 'outfile' on every iteration.
    """

    global ITERATION

    ITERATION += 1

    # I don't understand this business with the n_estimators -SB
    # if 'n_estimators' in hyperparameters:
    #     del hyperparameters['n_estimators']

    # subsample = hyperparameters['boosting_type'].get('subsample', 1.0)

    # hyperparameters['boosting_type'] = hyperparameters['boosting_type']['boosting_type']
    # hyperparameters['subsample'] = subsample

    for parameter_name in ['n_estimators', 'max_depth']: # whatever parameters need to be integers
        hyperparameters[parameter_name] = int(hyperparameters[parameter_name])

    start = timer()

    xgb_model = XGBRegressor(**hyperparameters) # or maybe **hyper...
    xgb_model.fit(train_features, train_labels)
    
    cross_val_scores = cross_val_score(xgb_model, train_features, train_labels, cv=N_FOLDS, scoring='neg_mean_squared_error')
    
    run_time = timer() - start

    best_score = -cross_val_scores.max()

    # boosting rounds that returned the highest cross val score
    # n_estimators = len(cross).......... ???

    of_connection = open(OUT_FILE, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([best_score, hyperparameters, ITERATION, run_time, best_score])
    of_connection.close()

    # Dictionary with information for evaluation
    return {'loss': best_score, 'hyperparameters': hyperparameters, 'iteration': ITERATION,
            'train_time': run_time, 'status': STATUS_OK}


In [180]:
from hyperopt import hp
from hyperopt.pyll.stochastic import sample

# learning_rate = {'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2))}

# Discrete uniform distribution
# num_leaves = {'num_leaves': hp.quniform('num_leaves', 30, 150, 1)}

space = {
    'max_depth': hp.quniform('max_depth', 2, 12, 1),
    'alpha': hp.choice('alpha', [0, 0.1]),
    'gamma': hp.uniform('gamma', 1,9),
    'booster': hp.choice('booster', ['gbtree', 'gblinear','dart']),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
    # 'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'n_estimators': hp.quniform('n_estimators', 1, 1000, 1),
}

x = sample(space)

print(x)

of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()

# Test the objective function
# results = objective(x)
# print('The cross validation loss = {:.5f}.'.format(results['loss']))
# print('The optimal number of estimators was {}.'.format(results['hyperparameters']['n_estimators']))

{'alpha': 0.1, 'booster': 'gbtree', 'gamma': 3.911065907354046, 'learning_rate': 0.06584646967679732, 'max_depth': 8.0, 'n_estimators': 762.0}


In [182]:
from hyperopt import tpe, fmin
tpe_algorithm = tpe.suggest

from hyperopt import Trials

# Record results
trials = Trials()

# Create a file and open a connection
OUT_FILE = 'bayes_test.csv'
of_connection = open(OUT_FILE, 'w')
writer = csv.writer(of_connection)

ITERATION = 0

global  ITERATION

ITERATION = 0

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, trials = trials,
            max_evals = 20)

best

# Write column names
headers = ['loss', 'hyperparameters', 'iteration', 'runtime', 'score']
writer.writerow(headers)
of_connection.close()

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

 10%|█         | 2/20 [22:36<3:57:34, 791.90s/trial, best loss: 680.3993530273438]

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:47:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:47:04] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:47:05] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:47:05] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



 30%|███       | 6/20 [23:43<29:07, 124.83s/trial, best loss: 669.3968505859375]  

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:48:09] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:48:10] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:48:11] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:48:12] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



...

 75%|███████▌  | 15/20 [25:01<01:18, 15.61s/trial, best loss: 669.3543701171875]

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:49:26] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:49:26] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:49:27] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



 80%|████████  | 16/20 [25:01<00:44, 11.04s/trial, best loss: 669.3543701171875]

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:49:27] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:49:27] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/home/steven/.pyenv/versions/.env/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:49:27] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "gamma", "max_depth" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



100%|██████████| 20/20 [36:31<00:00, 109.58s/trial, best loss: 669.3543701171875]


In [156]:
best

{'gamma': np.float64(6.071378912516411),
 'learning_rate': np.float64(0.012223406544371523),
 'max_depth': np.float64(11.0),
 'n_estimators': np.float64(32.0),
 'num_leaves': np.float64(140.0)}

In [183]:
best

{'alpha': np.int64(1),
 'booster': np.int64(1),
 'gamma': np.float64(5.5548154682628965),
 'learning_rate': np.float64(0.18497486356047765),
 'max_depth': np.float64(3.0),
 'n_estimators': np.float64(82.0)}